In [1]:
!pip3 install xgboost

In [2]:
from rf_dataset import RFDataset
import pandas as pd
import xgboost

import numpy as np

In [3]:
data = pd.read_csv('../assets/data.csv')
data.DATE = pd.to_datetime(data.DATE)

# Define features and target
features = [
    'CLOSE', 'VOLUME', 'HIGH', 'VOLATILITY_90D', 'CLOSE_SMA_3D', 'CLOSE_EWMA_3D', 
    'VOLATILITY_90D_SMA_3D', 'VOLATILITY_90D_EWMA_3D', 'CLOSE_SMA_9D', 'CLOSE_EWMA_9D', 
    'VOLATILITY_90D_SMA_9D', 'VOLATILITY_90D_EWMA_9D', 'CLOSE_SMA_21D', 'CLOSE_EWMA_21D', 
    'VOLATILITY_90D_SMA_21D', 'VOLATILITY_90D_EWMA_21D', 'CLOSE_SMA_50D', 'CLOSE_EWMA_50D', 
    'VOLATILITY_90D_SMA_50D', 'VOLATILITY_90D_EWMA_50D', 'CLOSE_SMA_65D', 'CLOSE_EWMA_65D', 
    'VOLATILITY_90D_SMA_65D', 'VOLATILITY_90D_EWMA_65D', 'CLOSE_SMA_120D', 'CLOSE_EWMA_120D', 
    'VOLATILITY_90D_SMA_120D', 'VOLATILITY_90D_EWMA_120D', 'CLOSE_SMA_360D', 'CLOSE_EWMA_360D', 
    'VOLATILITY_90D_SMA_360D', 'VOLATILITY_90D_EWMA_360D', 'RETURNS', 'CLOSE_ROC_3D', 
    'HIGH_ROC_3D', 'VOLATILITY_90D_ROC_3D', 'CLOSE_ROC_9D', 'HIGH_ROC_9D', 'VOLATILITY_90D_ROC_9D', 
    'CLOSE_ROC_21D', 'HIGH_ROC_21D', 'VOLATILITY_90D_ROC_21D', 'CLOSE_ROC_50D', 'HIGH_ROC_50D', 
    'VOLATILITY_90D_ROC_50D', 'CLOSE_ROC_65D', 'HIGH_ROC_65D', 'VOLATILITY_90D_ROC_65D', 
    'CLOSE_ROC_120D', 'HIGH_ROC_120D', 'VOLATILITY_90D_ROC_120D', 'CLOSE_ROC_360D', 'HIGH_ROC_360D', 
    'VOLATILITY_90D_ROC_360D'
]
target = 'RETURNS'
stock_ids = ['AAPL','NVDA','JNJ']
start_date = '2010-01-01'
end_date = '2023-12-28'
test_start_date = '2024-01-02'

In [4]:
rf_dataset = RFDataset(dataframe=data, stock_ids=stock_ids, start_date=start_date, end_date=end_date, features=features, horizon=2, tau=1e-6)

/Users/hassan/Desktop/research/randomforestlib/rf_dataset.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['future_returns'] = stock_data['RETURNS'].diff(periods=self.horizon)
/Users/hassan/Desktop/research/randomforestlib/rf_dataset.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['momentum'] = stock_data['RETURNS'].rolling(window=self.horizon).mean()
/Users/hassan/Desktop/research/randomforestlib/rf_dataset.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [5]:
x,xt,y,yt = rf_dataset.train_test_split_and_scale(test_start_date)

In [6]:
yt.value_counts()/np.sum(yt.value_counts().values) , y.value_counts()/np.sum(y.value_counts().values)

(label
 1    0.635659
 0    0.197674
 2    0.166667
 Name: count, dtype: float64,
 label
 0.0    0.333333
 1.0    0.333333
 2.0    0.333333
 Name: count, dtype: float64)

In [7]:
from model_training import train_xgboost_model, tune_xgboost_hyperparameters

In [8]:
len(x), len(y)

(3312, 3312)

In [9]:
m=train_xgboost_model(X_train=x,y_train=y)

In [10]:
m

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [14]:
xgb_param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 75, 100, 200, 300]
}

xgb_best_params = tune_xgboost_hyperparameters(x, y, xgb_param_grid)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
 

[CV] END ..learning_rate=0.001, max_depth=3, n_estimators=50; total time=   1.7s
[CV] END ..learning_rate=0.001, max_depth=3, n_estimators=50; total time=   1.7s
[CV] END ..learning_rate=0.001, max_depth=3, n_estimators=50; total time=   1.7s


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
 

[CV] END ..learning_rate=0.001, max_depth=3, n_estimators=75; total time=   2.6s


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
 

[CV] END ....learning_rate=0.2, max_depth=5, n_estimators=50; total time=   2.5s


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
 

[CV] END ....learning_rate=0.2, max_depth=5, n_estimators=50; total time=   2.6s


KeyboardInterrupt: 

In [12]:
from inference import compute_backtest
xgb_model, xgb_accuracy, xgb_X_test, xgb_y_test, xgb_y_pred = compute_backtest(rf_dataset, 'xgboost', xgb_best_params, test_start_date)
print(f"XGBoost Accuracy: {xgb_accuracy:.2f}")

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)
         CLOSE    VOLUME      HIGH  VOLATILITY_90D  CLOSE_SMA_3D  \
2996  1.178427  0.014489  1.184700        0.503549      1.278652   
2997  1.160854 -0.177270  1.152782        0.509633      1.246907   
2998  1.131272 -0.069188  1.11796